In [ ]:
# This example shows how to invoke SageMaker Endpoint from outside of AWS environment using SageMaker SDK
# Endpoint: XGBoost - Kaggle Bike Rental - Regressor Trained in XGBoost Lectures
# Makesure Endpoint is deployed before running this example
# 
# Reference:
#  https://github.com/awslabs/amazon-sagemaker-examples

# NOTE: SDK now requires additional permissions DescribeEndpoint, DescribeEndpointConfig in-addition to InvokeEndpoint
#   Please update SageMakerInvokeEndpoint permissions to reflect this policy document:
#   Logon with my_admin account and update permissions (IAM->Policies->SageMakerInvokeEndpoint->Edit Policy)
#   
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "sagemaker:DescribeEndpointConfig",
                "sagemaker:DescribeEndpoint",
                "sagemaker:InvokeEndpoint"
            ],
            "Resource": "*"
        }
    ]
}

In [ ]:
import boto3
import sagemaker
import math
import dateutil

from sagemaker.predictor import csv_serializer, json_deserializer, json_serializer, csv_deserializer

In [ ]:
# Establish a session with AWS
# Specify credentials and region to be used for this session.
# We will use a ml_user_predict credentials that has limited privileges
boto_session = boto3.Session(profile_name='ml_user_predict',region_name='us-east-1')

In [ ]:
sess = sagemaker.Session(boto_session=boto_session)

In [ ]:
# Get RealTimePredictor using SageMaker SDK
# Specify Your Endpoint Name
endpoint_name = 'xgboost-biketrain-v1'

predictor = sagemaker.predictor.RealTimePredictor(endpoint=endpoint_name,
                                                 sagemaker_session=sess)

In [ ]:
# We are sending data for inference in CSV format
predictor.content_type = 'text/csv'
predictor.serializer = csv_serializer
predictor.deserializer = None

In [ ]:
#datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
# Actual=562
sample_one = '2012-12-19 17:00:00,4,0,1,1,16.4,20.455,50,26.0027'
# Actual=569
sample_two = '2012-12-19 18:00:00,4,0,1,1,15.58,19.695,50,23.9994'
# Actual=4
sample_three = '2012-12-10 01:00:00,4,0,1,2,14.76,18.94,100,0'

In [ ]:
# Try invoking predict method
# It won't work...because we had transformed the original data to replace date time with a set of categorical features
predictor.predict(sample_one)

In [ ]:
# Raw Data Structure: 
# datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count

# Model expects data in this format (it was trained with these features):
# season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,month,day,dayofweek,hour

def transform_data(data):
    features = data.split(',')
    
    # Extract year, month, day, dayofweek, hour
    dt = dateutil.parser.parse(features[0])

    features.append(str(dt.year))
    features.append(str(dt.month))
    features.append(str(dt.day))
    features.append(str(dt.weekday()))
    features.append(str(dt.hour))
    
    # Return the transformed data. skip datetime field
    return ','.join(features[1:])

In [ ]:
print('Raw Data:\n',sample_one)
print('Transformed Data:\n',transform_data(sample_one))

In [ ]:
# Let's invoke prediction now
# Actual=562
predictor.predict(transform_data(sample_one))

In [ ]:
# Actual=569
predictor.predict(transform_data(sample_two))

In [ ]:
# Actual=4
predictor.predict(transform_data(sample_three))

In [ ]:
# Actual Count is 562...but predicted is 6.36.

# Model was trained with log1p(count)
# So, we need to apply inverse transformation to get the actual count
# Predicted Count looks much better now
result = predictor.predict(transform_data(sample_one))
result = result.decode("utf-8")
print ('Predicted Count', math.expm1(float(result)))

In [ ]:
result

In [ ]:
result = predictor.predict([transform_data(sample_one), transform_data(sample_two)])

In [ ]:
result.decode("utf-8")

In [ ]:
# Batch Prediction
# Transform data and invoke prediction in specified batch sizes
def run_predictions(data, batch_size):
    predictions = []
    
    transformed_data = [transform_data(row.strip()) for row in data]
    
    for i in range(0, len(data), batch_size):
        
        print(i,i+batch_size)
        
        result = predictor.predict(transformed_data[i : i + batch_size])
        
        result = result.decode("utf-8")
        result = result.split(',')
        
        predictions += [math.expm1(float(r)) for r in result]
                
    return predictions

In [ ]:
run_predictions([sample_one,sample_two,sample_three],10)

In [ ]:
# Run a batch prediction on Test.CSV File
# Read the file content
data = []
with open('test.csv','r') as f:
    # skip header
    f.readline()
    # Read remaining lines
    data = f.readlines()

In [ ]:
len(data)

In [ ]:
%%time
predictions = run_predictions(data,500)

In [ ]:
len(predictions),len(data)